In [1]:
%pip install plasTeX

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/722.0 kB ? eta -:--:--
     - ----------------------------------- 20.5/722.0 kB 640.0 kB/s eta 0:00:02
     --- --------------------------------- 71.7/722.0 kB 975.2 kB/s eta 0:00:01
     ----- ------------------------------ 112.6/722.0 kB 930.9 kB/s eta 0:00:01
     -------- --------------------------- 163.8/722.0 kB 893.0 kB/s eta 0:00:01
     ---------- ------------------------- 215.0/722.0 kB 935.2 kB/s eta 0:00:01
     ------------ ----------------------- 256.0/722.0 kB 983.0 kB/s eta 0:00:01
     -------------- --------------------- 297.0/722.0 kB 916.0 kB/s eta 0:00:01
     --------------- -------------------- 317.4/722.0 kB 936.6 kB/s eta 0:00:01
     ------------------ ----------------- 368.6/722.0 kB 880.6 kB/s eta 0:00:01
     -------------------- --------------- 409.6/722.0 kB 881.6 kB/s eta 0:00:01
     --------------------- -------------- 430.1/722.0 kB 865.6 kB/s

In [13]:
import re

# 示例 LaTeX 文本
latex_document = r"""
\documentclass{article}
\begin{document}

Refer to equation \eqref{massenergy} and \ref{pythagoras}.

Here is an equation without a label:
\begin{equation}
E = mc^2
\end{equation}

\begin{equation}
E = mc^2
\label{eq:mc2}\end{equation}

\begin{equation} \label{eq:mc3}
E = mc^3
\end{equation}

\begin{equation*}
E = mc^4
\label{eq:mc4}
\end{equation*}

\begin{equation}
\label{eq:second}
E = mc^5
\end{equation}

Another equation:
\begin{subequations}
\begin{align}
\label{eq:third}
a^2 + b^2 &= c^2 \\
E &= mc^2
\end{align}
\end{subequations}

\end{document}
"""

# 正则表达式来匹配 equation 环境
math_environments = ['equation', 'gather']
pattern = "|".join(rf"\\begin\{{{env}\*?\}}\s*(\\label\{{[^}}]*\}})?(.*?)(\\label\{{[^}}]*\}})?\s*\\end\{{{env}\*?\}}" for env in math_environments)

# 使用 re.DOTALL 使 '.' 特殊字符匹配任何字符包括换行符
matches = re.finditer(pattern, latex_document, re.DOTALL)
label_count = 1

# 检查每个方程和标签
updated_latex_document = latex_document
replacement_deltas = 0  # Keep track of the changes in string length due to replacements

for match in matches:
    pre_label = match.group(1)
    content = match.group(2)
    post_label = match.group(3)
    start, end = match.span()
    # print(f'pre_label {pre_label} content {content} post_label {post_label}')
    # print(match.group(), len(match.group()))
    # Adjust the replacement range by the accumulated deltas
    start += replacement_deltas
    end += replacement_deltas

    if not pre_label and not post_label:
        print(f"Found unlabeled environment: {content.strip()}")
        new_label = f"\\label{{eq:{label_count}}}"
        label_count += 1
        new_content = f"\\begin{{equation}} {new_label}\n{content.strip()} \n\\end{{equation}}"
        updated_latex_document = updated_latex_document[:start] + new_content + updated_latex_document[end:]
        replacement_deltas += len(new_content) - (end - start)  # Update the delta

# 处理 subequations 环境

env = "subequations"
subequations_pattern = rf"\\begin\{{{env}\*?\}}\s*(\\label\{{[^}}]*\}})?(.*?)(\\label\{{[^}}]*\}})?\s*\\end\{{{env}\*?\}}"
align_pattern = r"\\begin\{align\}(.*?)\\end\{align\}"
matches = re.finditer(subequations_pattern, updated_latex_document, re.DOTALL)

for match in matches:
    pre_label = match.group(1)
    content = match.group(2)
    post_label = match.group(3)
    start, end = match.span()
    print(pre_label, content, post_label)
    # print('Found subequations block:', subequations_block.strip())
    if not pre_label and not post_label:
        print(f"Found unlabeled environment: {content.strip()}")
        new_label = f"\\label{{eq:{label_count}}}"
        label_count += 1
        new_content = f"\\begin{{subequations}} {new_label}\n{content.strip()} \n\\end{{subequations}}"
        updated_latex_document = updated_latex_document[:start] + new_content + updated_latex_document[end:]
        replacement_deltas += len(new_content) - (end - start)  # Update the delta
    # # 为 subequations 添加标签（如果没有的话）
    # if "\label{" not in subequations_block:
    #     label_insert = f"\\label{{eq:{label_count}}}"
    #     label_count += 1
    #     replacement = f"\\begin{{subequations}}{label_insert}\n{align_content}\\end{{subequations}}"
    #     print(replacement)
    # else:
    #     replacement = subequations_block

    # # 更新文档内容
    # start += replacement_deltas
    # end += replacement_deltas
    # updated_latex_document = updated_latex_document[:start] + replacement + updated_latex_document[end:]
    # replacement_deltas += len(replacement) - len(subequations_block)

print('=============****')
# 显示或保存更新后的 LaTeX 文档
# print(updated_latex_document)


Found unlabeled environment: E = mc^2
None \begin{align}
\label{eq:third}
a^2 + b^2 &= c^2 \\
E &= mc^2
\end{align} None
Found unlabeled environment: \begin{align}
\label{eq:third}
a^2 + b^2 &= c^2 \\
E &= mc^2
\end{align}
=============****
